In [27]:
from IPython.core.display import display, HTML
import pandas as pd
import numpy as np
import os, sys
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# BASELINE
# TFIDF
# SICK_VERBADJNOUN_lemma	0.611	0.091		# TFIDF
# SICK_ADJNOUN_lemma	0.558	0.111			# TFIDF
# SICK_VERBADJNOUN_orth	0.568	0.106		# TFIDF
# SICK_ADJNOUN_orth	0.542	0.119			# TFIDF

In [16]:
sick_test = pd.DataFrame.from_csv('SA_results/SICK_test_annotated.txt', sep='\t', header=0)
sick_test_overlap = pd.DataFrame.from_csv('SA_results/semeval_overlap_test.tsv', sep='\t', header=0)
sick_test['relatedness_score'].corr(sick_test_overlap['relatedness_score'])
#0.62701073200106172

0.62701073200106172

In [34]:
results_all = pd.DataFrame.from_csv('SA_results/scores_all.tsv', sep='\t', index_col=['crossfold_id','dev_file_index'])
results_all['edge_available'] = results_all.apply(lambda row: False if 'blanked' in row.train_data_path else True, axis=1)
results_all['order_aware'] = results_all.apply(lambda row: True if 'LSTM' in row.tree_embedder else False, axis=1)
#results_all.set_index(['crossfold_id','dev_file_index'])
results_all

clipping      optimizer  early_stop_queue  \
crossfold_id dev_file_index                                              
0            0                      5  AdamOptimizer                25   
             1                      5  AdamOptimizer                25   
             2                      5  AdamOptimizer                25   
             3                      5  AdamOptimizer                25   
             4                      5  AdamOptimizer                25   
1            0                      5  AdamOptimizer                25   
             1                      5  AdamOptimizer                25   
             2                      5  AdamOptimizer                25   
             3                      5  AdamOptimizer                25   
             4                      5  AdamOptimizer                25   
2            0                      5  AdamOptimizer                25   
             1                      5  AdamOptimizer                25   
             2                      5  AdamOptimizer                25   
             3                      5  AdamOptimizer                25   
             4                      5  AdamOptimizer                25   
3            0                      5  AdamOptimizer                25   
             1                      5  AdamOptimizer                25   
             2                      5  AdamOptimizer                25   
             3                      5  AdamOptimizer                25   
             4                      5  AdamOptimizer                25   
4            0                      5  AdamOptimizer                25   
             1                      5  AdamOptimizer                25   
             2                      5  AdamOptimizer                25   
             3                      5  AdamOptimizer                25   
             4                      5  AdamOptimizer                25   
5            0                      5  AdamOptimizer                25   
             1                      5  AdamOptimizer                25   
             2                      5  AdamOptimizer                25   
             3                      5  AdamOptimizer                25   
             4                      5  AdamOptimizer                25   
...                               ...            ...               ...   
4            0                      5  AdamOptimizer                25   
             1                      5  AdamOptimizer                25   
             2                      5  AdamOptimizer                25   
             3                      5  AdamOptimizer                25   
             4                      5  AdamOptimizer                25   
5            0                      5  AdamOptimizer                25   
             1                      5  AdamOptimizer                25   
             2                      5  AdamOptimizer                25   
             3                      5  AdamOptimizer                25   
             4                      5  AdamOptimizer                25   
6            0                      5  AdamOptimizer                25   
             1                      5  AdamOptimizer                25   
             2                      5  AdamOptimizer                25   
             3                      5  AdamOptimizer                25   
             4                      5  AdamOptimizer                25   
7            0                      5  AdamOptimizer                25   
             1                      5  AdamOptimizer                25   
             2                      5  AdamOptimizer                25   
             3                      5  AdamOptimizer                25   
             4                      5  AdamOptimizer                25   
8            0                      5  AdamOptimizer                25   
             1                   

In [37]:
res = results_all[['edge_available', 'order_aware', 'score_dev_best', 'score_test']]
res

edge_available  order_aware  score_dev_best  \
crossfold_id dev_file_index                                                
0            0                         True         True        0.844828   
             1                         True         True        0.843773   
             2                         True         True        0.858302   
             3                         True         True        0.835662   
             4                         True         True        0.860294   
1            0                         True         True        0.830682   
             1                         True         True        0.841132   
             2                         True         True        0.853065   
             3                         True         True        0.839592   
             4                         True         True        0.863516   
2            0                         True         True        0.842043   
             1                         True         True        0.840754   
             2                         True         True        0.861061   
             3                         True         True        0.842728   
             4                         True         True        0.865565   
3            0                         True         True        0.838747   
             1                         True         True        0.833676   
             2                         True         True        0.861122   
             3                         True         True        0.843764   
             4                         True         True        0.869045   
4            0                         True         True        0.839826   
             1                         True         True        0.852424   
             2                         True         True        0.861173   
             3                         True         True        0.836926   
             4                         True         True        0.866512   
5            0                         True         True        0.836645   
             1                         True         True        0.843071   
             2                         True         True        0.859388   
             3                         True         True        0.843120   
             4                         True         True        0.864658   
...                                     ...          ...             ...   
4            0                        False        False        0.842234   
             1                        False        False        0.835141   
             2                        False        False        0.856935   
             3                        False        False        0.833710   
             4                        False        False        0.866730   
5            0                        False        False        0.845410   
             1                        False        False        0.837387   
             2                        False        False        0.855054   
             3                        False        False        0.832271   
             4                        False        False        0.867368   
6            0                        False        False        0.844674   
             1                        False        False        0.835141   
             2                        False        False        0.855665   
             3                        False        False        0.833073   
             4                        False        False        0.866818   
7            0                        False        False        0.846333   
             1                        False        False        0.834447   
             2                        False        False        0.855733   
             3                        False        False        0.829912   
             4                        False        False        0.868060   
8

In [42]:
res[res.edge_available==False]

edge_available  order_aware  score_dev_best  \
crossfold_id dev_file_index                                                
0            0                        False         True        0.839746   
             1                        False         True        0.843276   
             2                        False         True        0.856579   
             3                        False         True        0.840963   
             4                        False         True        0.874278   
1            0                        False         True        0.841049   
             1                        False         True        0.840956   
             2                        False         True        0.859921   
             3                        False         True        0.836986   
             4                        False         True        0.858078   
2            0                        False         True        0.834081   
             1                        False         True        0.837815   
             2                        False         True        0.859024   
             3                        False         True        0.842252   
             4                        False         True        0.872087   
3            0                        False         True        0.832256   
             1                        False         True        0.845399   
             2                        False         True        0.857663   
             3                        False         True        0.847024   
             4                        False         True        0.865768   
4            0                        False         True        0.834691   
             1                        False         True        0.840894   
             2                        False         True        0.858695   
             3                        False         True        0.843855   
             4                        False         True        0.865591   
5            0                        False         True        0.832326   
             1                        False         True        0.847299   
             2                        False         True        0.860898   
             3                        False         True        0.837631   
             4                        False         True        0.865859   
...                                     ...          ...             ...   
4            0                        False        False        0.842234   
             1                        False        False        0.835141   
             2                        False        False        0.856935   
             3                        False        False        0.833710   
             4                        False        False        0.866730   
5            0                        False        False        0.845410   
             1                        False        False        0.837387   
             2                        False        False        0.855054   
             3                        False        False        0.832271   
             4                        False        False        0.867368   
6            0                        False        False        0.844674   
             1                        False        False        0.835141   
             2                        False        False        0.855665   
             3                        False        False        0.833073   
             4                        False        False        0.866818   
7            0                        False        False        0.846333   
             1                        False        False        0.834447   
             2                        False        False        0.855733   
             3                        False        False        0.829912   
             4                        False        False        0.868060   
8